In [1]:

import sys
!{sys.executable} -m pip install flask flask_sqlalchemy
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

# Initialize Flask app
app = Flask(__name__)

# SQLite database setup
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///blog.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Blog Post Model
class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(200), nullable=False)
    content = db.Column(db.Text, nullable=False)
    category = db.Column(db.String(100), nullable=False)
    tags = db.Column(db.String(200), nullable=True)  # stored as comma-separated
    createdAt = db.Column(db.DateTime, default=datetime.utcnow)
    updatedAt = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "content": self.content,
            "category": self.category,
            "tags": self.tags.split(",") if self.tags else [],
            "createdAt": self.createdAt.isoformat(),
            "updatedAt": self.updatedAt.isoformat()
        }

# Create database tables
with app.app_context():
    db.create_all()

# ---------------- CRUD Endpoints ----------------

# Create Blog Post
@app.route('/posts', methods=['POST'])
def create_post():
    data = request.json
    if not data or not data.get("title") or not data.get("content") or not data.get("category"):
        return jsonify({"error": "Missing required fields"}), 400

    new_post = Post(
        title=data["title"],
        content=data["content"],
        category=data["category"],
        tags=",".join(data.get("tags", []))
    )
    db.session.add(new_post)
    db.session.commit()
    return jsonify(new_post.to_dict()), 201

# Get All Blog Posts (with optional search)
@app.route('/posts', methods=['GET'])
def get_posts():
    term = request.args.get("term")
    query = Post.query
    if term:
        search = f"%{term}%"
        query = query.filter((Post.title.like(search)) | (Post.content.like(search)) | (Post.category.like(search)))
    posts = query.all()
    return jsonify([post.to_dict() for post in posts]), 200

# Get Single Blog Post
@app.route('/posts/<int:post_id>', methods=['GET'])
def get_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        return jsonify({"error": "Post not found"}), 404
    return jsonify(post.to_dict()), 200

# Update Blog Post
@app.route('/posts/<int:post_id>', methods=['PUT'])
def update_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        return jsonify({"error": "Post not found"}), 404

    data = request.json
    if not data or not data.get("title") or not data.get("content") or not data.get("category"):
        return jsonify({"error": "Missing required fields"}), 400

    post.title = data["title"]
    post.content = data["content"]
    post.category = data["category"]
    post.tags = ",".join(data.get("tags", []))
    db.session.commit()
    return jsonify(post.to_dict()), 200

# Delete Blog Post
@app.route('/posts/<int:post_id>', methods=['DELETE'])
def delete_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        return jsonify({"error": "Post not found"}), 404
    db.session.delete(post)
    db.session.commit()
    return "", 204


from werkzeug.serving import run_simple

def run_app():
    run_simple('localhost', 5000, app, use_reloader=False)


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import sys
print(sys.executable)


C:\ProgramData\anaconda3\python.exe


In [2]:
!C:\ProgramData\anaconda3\python.exe -m pip install flask_sqlalchemy


Defaulting to user installation because normal site-packages is not writeable
